In [15]:
install.packages("stringr")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [100]:
install.packages("dplyr")

also installing the dependencies ‘ellipsis’, ‘pillar’, ‘glue’, ‘lifecycle’, ‘rlang’, ‘tibble’, ‘tidyselect’, ‘vctrs’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [101]:
library(stringr)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
p1 <- read.csv("Planilha_Séries_Temporais_data (16).csv")

Warning message in file(file, "rt"):
“cannot open file 'Planilha_Séries_Temporais_data (16).csv': No such file or directory”

ERROR: Error in file(file, "rt"): cannot open the connection


In [36]:

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

library("aws.s3")
obj <- get_object(
    object = "Planilha_Séries_Temporais_data (16).csv",
    bucket = "voprampeliminationsimulation-donotdelete-pr-0zwiwtgxy7dqsm",
    key = "b1c2443ccfc04d278887b96652f3e13a",
    secret = "4e9df613cc37174ea596c447b546a7981ef086094a75049f",
    check_region = FALSE,
    base_url = "s3-api.us-geo.objectstorage.service.networklayer.com")

p1 <- read.csv(text = rawToChar(obj), sep = ";")
head(p1)

Contexto,Data.e.Hora,var_alias,VAR_VALUE
BP-7046-FS || 49,04/08/20 09:43:44.100,NÚMERO CORRENTE DE SOLDAGEM,39701
BP-7046-FS || 49,04/08/20 09:43:44.300,NÚMERO CORRENTE DE SOLDAGEM,39701
BP-7046-FS || 49,04/08/20 09:43:44.400,NÚMERO CORRENTE DE SOLDAGEM,39701
BP-7046-FS || 49,04/08/20 09:43:44.500,NÚMERO CORRENTE DE SOLDAGEM,39701
BP-7046-FS || 49,04/08/20 09:43:44.600,NÚMERO CORRENTE DE SOLDAGEM,39701
BP-7046-FS || 49,04/08/20 09:43:44.700,NÚMERO CORRENTE DE SOLDAGEM,39701


In [102]:
p <- p1
attach(p)
VAR_VALUE <- str_replace(VAR_VALUE, ",",".")
VAR_VALUE <- as.numeric(VAR_VALUE)
Data.e.Hora <- str_replace(Data.e.Hora, "/","-")

In [103]:
Data.e.Hora <- str_replace_all(Data.e.Hora, "/","-")
Data.e.Hora <- as.POSIXlt(Data.e.Hora, "%Y-%m-%d %H:%M:%OS")
Data.e.Hora <- format(Data.e.Hora, "%Y-%m-%d %H:%M:%OS3")

In [104]:
p <- arrange(p,Data.e.Hora)

In [107]:
v <- filter(p, var_alias == "VELOCIDADE ALIMENTAÇÃO FIO SOLDA")
n <- filter(p, var_alias == "NÚMERO CORRENTE DE SOLDAGEM")

In [115]:
p <- inner_join(v,n,by="Data.e.Hora")
p <- select(p,c("Contexto.x", "Data.e.Hora", "VAR_VALUE.x", "VAR_VALUE.y"))

Contexto.x,Data.e.Hora,VAR_VALUE.x,VAR_VALUE.y
BP-7046-FS || 48,04/08/20 09:43:39.300,"4,3",39700
BP-7046-FS || 48,04/08/20 09:43:39.400,"7,9",39700
BP-7046-FS || 48,04/08/20 09:43:39.500,"11,4",39700
BP-7046-FS || 48,04/08/20 09:43:39.600,12,39700
BP-7046-FS || 48,04/08/20 09:43:39.700,"12,3",39700
BP-7046-FS || 48,04/08/20 09:43:39.800,"12,4",39700


In [118]:
inc <- numeric(nrow(p))
inc[1] <- 1
for(i in 2:nrow(p)){
    if(p[i,4] != p[i-1,4]){
        inc[i] <- 1
        inc[i+1] <- 2
    } else if(inc[i] != 2){
        inc[i] <- 0
    }
}

p <- cbind(p, inc)

In [121]:
p2 <- filter(p,inc == 2)
head(p2)

Contexto.x,Data.e.Hora,VAR_VALUE.x,VAR_VALUE.y,inc
BP-7046-FS || 49,04/08/20 09:43:44.300,"11,9",39701,2
BP-7046-FS || 49,04/08/20 09:43:51.100,"3,5",39702,2
BP-7046-FS || 49,04/08/20 09:43:53.700,"3,6",39703,2
BP-7046-FS || 49,04/08/20 09:43:56.600,"7,1",39704,2
BP-7046-FS || 45,04/08/20 09:44:17.400,"11,2",39705,2
BP-7046-FS || 45,04/08/20 09:44:24.900,"4,4",39706,2
